# Overview
We will building out an agent to sythesize news and econmic data from various sources and provide the user with a general overview of the market sentiment.

In [18]:
from openai import OpenAI
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",
)

model = "gpt-oss:latest"

In [19]:
import yfinance as yf
import time

def get_market_performance():
    """
    Gets S&P 500, QQQ, and VIX performance metrics
    Returns: dict with last close and weekly percent changes
    """
    tickers_info = {
        'S&P 500': '^GSPC',
        'QQQ': 'QQQ',
        'VIX': '^VIX'
    }
    
    results = {}
    
    print("Fetching market data from Yahoo Finance...\n")
    
    for name, symbol in tickers_info.items():
        try:
            ticker = yf.Ticker(symbol)
            hist = ticker.history(period="1mo")
            
            # Remove NaN rows
            hist = hist.dropna()
            
            if len(hist) >= 2:
                # Last close (most recent day)
                last_close = hist['Close'].iloc[-1]
                prev_close = hist['Close'].iloc[-2]
                daily_change = ((last_close - prev_close) / prev_close) * 100
                
                # Weekly change (last 5-7 trading days)
                if len(hist) >= 7:
                    week_ago_close = hist['Close'].iloc[-6]  # ~5 trading days
                    weekly_change = ((last_close - week_ago_close) / week_ago_close) * 100
                else:
                    weekly_change = None
                
                results[name] = {
                    'last_close': round(last_close, 2),
                    'daily_change': round(daily_change, 2),
                    'weekly_change': round(weekly_change, 2) if weekly_change else None
                }
                
                print(f"✓ {name}: ${last_close:.2f} | Daily: {daily_change:+.2f}% | Weekly: {weekly_change:+.2f}%" if weekly_change else f"✓ {name}: ${last_close:.2f} | Daily: {daily_change:+.2f}%")
            else:
                print(f"⚠ {name}: Not enough data")
                results[name] = None
            
            # Small delay to avoid rate limiting
            time.sleep(0.5)
                
        except Exception as e:
            print(f"❌ {name}: {str(e)[:60]}")
            results[name] = None
    
    print("\n✓ Market data fetch complete!")
    return results

market_data = get_market_performance()


Fetching market data from Yahoo Finance...

✓ S&P 500: $6602.99 | Daily: +0.98% | Weekly: -1.95%
✓ QQQ: $590.07 | Daily: +0.75% | Weekly: -3.09%
✓ VIX: $23.43 | Daily: -11.32% | Weekly: +18.15%

✓ Market data fetch complete!


In [20]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [21]:
def get_fear_greed_index():
    url = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"

    r = requests.get(url, headers=headers)
    score = r.json()['fear_and_greed']['score']
    rating = r.json()['fear_and_greed']['rating']

    return score, rating

[score, rating] = get_fear_greed_index()
fear_and_greed_index = ('Fear and Greed Index Score:', score, '\nSentiment:', rating)
print(fear_and_greed_index)

('Fear and Greed Index Score:', 10.5142857142857, '\nSentiment:', 'extreme fear')


In [22]:
# Combine all market data for analysis
market_summary = f"""
MARKET SENTIMENT DATA:

Fear & Greed Index:
- Score: {score}
- Rating: {rating}
"""

# Check if we have any market performance data
has_market_data = any(data is not None for data in market_data.values())

if has_market_data:
    market_summary += "\n\nMarket Performance:"
    for name, data in market_data.items():
        if data:
            market_summary += f"\n{name}:"
            market_summary += f"\n  Last Close: ${data['last_close']}"
            market_summary += f"\n  Daily Change: {data['daily_change']:+.2f}%"
            if data['weekly_change']:
                market_summary += f"\n  Weekly Change: {data['weekly_change']:+.2f}%"
else:
    market_summary += "\n\nNote: Market performance data temporarily unavailable."

print("=" * 60)
print("Sending to AI for analysis:")
print(market_summary)
print("=" * 60)

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are an expert financial analyst that provides comprehensive market sentiment analysis. Analyze the data provided and give actionable insights about current market conditions. If market performance data is unavailable, focus your analysis on the Fear & Greed Index."},
        {"role": "user", "content": market_summary},
    ],
)

print("\n" + "=" * 60)
print("AI MARKET SENTIMENT ANALYSIS:")
print("=" * 60)
print(response.choices[0].message.content)


Sending to AI for analysis:

MARKET SENTIMENT DATA:

Fear & Greed Index:
- Score: 10.5142857142857
- Rating: extreme fear


Market Performance:
S&P 500:
  Last Close: $6602.99
  Daily Change: +0.98%
  Weekly Change: -1.95%
QQQ:
  Last Close: $590.07
  Daily Change: +0.75%
  Weekly Change: -3.09%
VIX:
  Last Close: $23.43
  Daily Change: -11.32%
  Weekly Change: +18.15%

AI MARKET SENTIMENT ANALYSIS:
## Market Sentiment Snapshot (as of today)

| Indicator | Value | Interpretation |
|-----------|-------|----------------|
| **Fear & Greed Index** | **10.5** | *Extreme fear* – a historically low reading that signals widespread pessimism and potentially oversold conditions. |
| **S&P 500** | ↑ 0.98 % today; ↓ 1.95 % this week | A modest intraday rally but a weak week‑to‑date trend. |
| **QQQ (NASDAQ‑100)** | ↑ 0.75 % today; ↓ 3.09 % this week | Similar pattern to the broader market – a small daily lift, but a stronger weekly downturn. |
| **VIX (CBOE Volatility Index)** | ↓ 11.3 % today; ↑ 